In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    path_to_file = '/content/gdrive/My Drive/CS5340_project/cs5340-covid-pgm/notebook/'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/CS5340_project/cs5340-covid-pgm/notebook/
/content/gdrive/My Drive/CS5340_project/cs5340-covid-pgm/notebook


In [2]:
!pip install pgmpy

     |████████████████████████████████| 1.9 MB 22.0 MB/s 


In [3]:
import pandas as pd
import numpy as np

In [4]:
import pickle
import pgmpy

In [5]:
with open("../models/bayesian_18countries_learnt_bn_treesearch.pickle", "rb") as handle:
  model = pickle.load(handle)

In [6]:
train_df = pd.read_csv("../data/processed/train_18_countries.csv")
print(train_df.shape)
train_df.head()

(495, 307)


,deaths_per_mil_cat_india,facial_covering_new_zealand,cancel_public_events_argentina,debt_relief_india,income_support_singapore,school_closures_argentina,vaccine_policy_china,restriction_internal_movement_indonesia,workplace_closures_finland,internation_travel_control_finland,...,cases_per_mil_cat_finland,cases_per_mil_cat_hong_kong,cases_per_mil_cat_indonesia,cases_per_mil_cat_india,cases_per_mil_cat_israel,cases_per_mil_cat_malaysia,cases_per_mil_cat_new_zealand,cases_per_mil_cat_singapore,cases_per_mil_cat_united_states,cases_per_mil_cat_south_africa
0,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,6,2
1,0,2,2,0.0,2,3,4,2,2,3.0,...,1,0,0,1,4,1,0,1,4,1
2,0,3,1,1.0,2,0,5,2,1,2.0,...,4,0,1,1,6,4,0,3,6,3
3,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,5,2
4,0,2,2,0.0,2,1,4,2,2,3.0,...,1,0,0,0,4,1,0,1,4,1


In [7]:
test_df = pd.read_csv("../data/processed/test_18_countries.csv")
print(test_df.shape)
test_df.head()

(165, 307)


,deaths_per_mil_cat_india,facial_covering_new_zealand,cancel_public_events_argentina,debt_relief_india,income_support_singapore,school_closures_argentina,vaccine_policy_china,restriction_internal_movement_indonesia,workplace_closures_finland,internation_travel_control_finland,...,cases_per_mil_cat_finland,cases_per_mil_cat_hong_kong,cases_per_mil_cat_indonesia,cases_per_mil_cat_india,cases_per_mil_cat_israel,cases_per_mil_cat_malaysia,cases_per_mil_cat_new_zealand,cases_per_mil_cat_singapore,cases_per_mil_cat_united_states,cases_per_mil_cat_south_africa
0,0,3,1,1.0,2,0,5,2,1,2.0,...,4,0,1,1,6,4,0,3,6,3
1,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,6,3,0,1,6,2
2,0,2,2,2.0,2,3,0,2,1,3.0,...,0,0,0,0,1,0,0,1,1,1
3,0,3,2,1.0,2,1,5,2,1,2.0,...,1,0,1,1,5,3,0,1,5,2
4,0,0,2,2.0,2,3,0,1,1,3.0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
from pgmpy.inference import VariableElimination

ve = VariableElimination(model)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# import networkx as nx
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1,1, figsize=(80,80))
# nx.draw(model, with_labels=True, ax=ax)
# plt.tight_layout()

In [11]:
test_df = test_df.astype(int)

In [12]:
cols_to_remove = list(set(test_df.columns) - set(model.nodes()))

In [13]:
cols_to_remove

[]

In [14]:
test_df.filter(regex="[a-z]_singapore")

,income_support_singapore,shn_requirement_singapore,contact_tracing_singapore,deaths_per_mil_cat_singapore,vaccine_policy_singapore,restriction_internal_movement_singapore,testing_policy_singapore,internation_travel_control_singapore,pi_campaign_singapore,school_closures_singapore,cancel_public_events_singapore,debt_relief_singapore,workplace_closures_singapore,restrict_public_gathering_singapore,closure_public_transport_singapore,facial_covering_singapore,cases_per_mil_cat_singapore
0,2,1,2,0,5,0,3,3,2,1,2,2,3,4,0,4,3
1,2,1,2,0,5,0,3,3,2,1,1,2,1,4,0,4,1
2,2,1,2,0,0,0,2,3,2,1,2,2,2,2,0,4,1
3,2,1,2,0,5,0,3,3,2,1,1,2,1,4,0,4,1
4,2,1,2,0,0,0,2,3,2,1,2,2,2,2,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2,1,2,0,0,0,2,3,2,1,1,2,2,2,0,4,1
161,2,1,2,0,5,0,3,3,2,2,1,2,1,4,0,4,1
162,2,1,2,0,2,0,3,3,2,1,1,2,2,4,0,4,1
163,1,2,2,0,0,2,2,3,2,3,2,2,3,2,0,4,0


In [15]:
cols_to_remove = cols_to_remove + list(test_df.filter(regex="[a-z]_singapore").columns)

In [16]:
malaysia_features = list(test_df.filter(regex="[a-z]_malaysia").columns)
print(malaysia_features)

['cancel_public_events_malaysia', 'facial_covering_malaysia', 'workplace_closures_malaysia', 'restrict_public_gathering_malaysia', 'restriction_internal_movement_malaysia', 'school_closures_malaysia', 'debt_relief_malaysia', 'contact_tracing_malaysia', 'vaccine_policy_malaysia', 'deaths_per_mil_cat_malaysia', 'income_support_malaysia', 'testing_policy_malaysia', 'closure_public_transport_malaysia', 'pi_campaign_malaysia', 'shn_requirement_malaysia', 'internation_travel_control_malaysia', 'cases_per_mil_cat_malaysia']


In [17]:
preds_df = pd.DataFrame()
for c in cols_to_remove:
  preds = []
  for idx, row in test_df.drop(cols_to_remove, axis = 1).iterrows():
    evidence = row[malaysia_features].to_dict()
    # evidence = {"date": row["date"]}
    # print(evidence)
    res = ve.query(
          [c], evidence=evidence, show_progress=False
      )
    ans_dict = {}
    for key, val in zip(res.state_names[res.variables[0]], res.values):
      ans_dict[key] = val
    max_val = max(ans_dict, key=ans_dict.get)
    preds.append(max_val)

  # predictions = []
  # for t in preds:
  #   predictions.append(t)
  preds_df[f"{c}_predicted"] = preds

In [18]:
preds_df.to_csv("2_7_predictions.csv", index=False)
preds_df

,income_support_singapore_predicted,shn_requirement_singapore_predicted,contact_tracing_singapore_predicted,deaths_per_mil_cat_singapore_predicted,vaccine_policy_singapore_predicted,restriction_internal_movement_singapore_predicted,testing_policy_singapore_predicted,internation_travel_control_singapore_predicted,pi_campaign_singapore_predicted,school_closures_singapore_predicted,cancel_public_events_singapore_predicted,debt_relief_singapore_predicted,workplace_closures_singapore_predicted,restrict_public_gathering_singapore_predicted,closure_public_transport_singapore_predicted,facial_covering_singapore_predicted,cases_per_mil_cat_singapore_predicted
0,2,1,2.0,0,5,0,3.0,2.0,2,1,1,2.0,1,4,0,4,3
1,2,1,2.0,0,5,0,3.0,3.0,2,1,1,2.0,2,4,0,4,1
2,2,1,2.0,0,0,0,2.0,3.0,2,1,1,2.0,2,2,0,4,1
3,2,1,2.0,0,5,0,3.0,3.0,2,1,1,2.0,2,4,0,4,1
4,2,1,2.0,0,0,0,2.0,3.0,2,1,1,2.0,2,2,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2,1,2.0,0,0,0,2.0,3.0,2,1,1,2.0,2,2,0,4,1
161,2,1,2.0,0,5,0,3.0,3.0,2,1,1,2.0,1,4,0,4,1
162,2,1,2.0,0,0,0,3.0,3.0,2,1,1,2.0,2,4,0,4,1
163,2,2,2.0,0,0,2,2.0,3.0,2,1,2,2.0,2,2,0,4,0


In [19]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

def compute_evaluation_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average="weighted"),
        "recall": recall_score(y_true, y_pred, average="weighted"),
    }

In [20]:
res = {}
for c in cols_to_remove:
  ypred = preds_df[f"{c}_predicted"].values
  ytrue = test_df[c].values
  res[c] = compute_evaluation_metrics(ytrue, ypred)

In [21]:
res

{'cancel_public_events_singapore': {'accuracy': 0.8363636363636363,
  'f1': 0.826207357362931,
  'recall': 0.8363636363636363},
 'cases_per_mil_cat_singapore': {'accuracy': 0.8787878787878788,
  'f1': 0.8546305260247743,
  'recall': 0.8787878787878788},
 'closure_public_transport_singapore': {'accuracy': 1.0,
  'f1': 1.0,
  'recall': 1.0},
 'contact_tracing_singapore': {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0},
 'deaths_per_mil_cat_singapore': {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0},
 'debt_relief_singapore': {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0},
 'facial_covering_singapore': {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0},
 'income_support_singapore': {'accuracy': 0.9696969696969697,
  'f1': 0.9547785547785548,
  'recall': 0.9696969696969697},
 'internation_travel_control_singapore': {'accuracy': 0.9151515151515152,
  'f1': 0.9195519019987104,
  'recall': 0.9151515151515152},
 'pi_campaign_singapore': {'accuracy': 1.0, 'f1': 1.0, 'recall': 1.0},
 'restrict_public_gathering_si

In [22]:
# Analysis
print(ve.query(
          ["workplace_closures_singapore"], evidence={"workplace_closures_malaysia": 3}, show_progress=False
      ))

+---------------------------------+-------------------------------------+
| workplace_closures_singapore    |   phi(workplace_closures_singapore) |
+=================================+=====================================+
| workplace_closures_singapore(1) |                              0.1154 |
+---------------------------------+-------------------------------------+
| workplace_closures_singapore(2) |                              0.8674 |
+---------------------------------+-------------------------------------+
| workplace_closures_singapore(3) |                              0.0173 |
+---------------------------------+-------------------------------------+


In [31]:
print(ve.query(
          ["vaccine_policy_singapore"], evidence={"vaccine_policy_malaysia": 1}, show_progress=False
      ))

+-----------------------------+---------------------------------+
| vaccine_policy_singapore    |   phi(vaccine_policy_singapore) |
+=============================+=================================+
| vaccine_policy_singapore(0) |                          0.1302 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(1) |                          0.0884 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(2) |                          0.2313 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(4) |                          0.4310 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(5) |                          0.1191 |
+-----------------------------+---------------------------------+


In [30]:
print(ve.query(
          ["vaccine_policy_singapore"], evidence={"vaccine_policy_malaysia": 5}, show_progress=False
      ))

+-----------------------------+---------------------------------+
| vaccine_policy_singapore    |   phi(vaccine_policy_singapore) |
+=============================+=================================+
| vaccine_policy_singapore(0) |                          0.0000 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(1) |                          0.0000 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(2) |                          0.0000 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(4) |                          0.0000 |
+-----------------------------+---------------------------------+
| vaccine_policy_singapore(5) |                          1.0000 |
+-----------------------------+---------------------------------+


In [23]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={"cases_per_mil_cat_malaysia": 5}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.3387 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.6613 |
+-------------------------------------------+---------------------------------------------+


In [24]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={"cases_per_mil_cat_malaysia": 4}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.3387 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.6613 |
+-------------------------------------------+---------------------------------------------+


In [25]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={"cases_per_mil_cat_malaysia": 3}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.3154 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.6846 |
+-------------------------------------------+---------------------------------------------+


In [26]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={"cases_per_mil_cat_malaysia": 2}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.2158 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.7842 |
+-------------------------------------------+---------------------------------------------+


In [27]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={"cases_per_mil_cat_malaysia": 1}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.1048 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.8952 |
+-------------------------------------------+---------------------------------------------+


In [28]:
print(ve.query(
          ["internation_travel_control_singapore"], evidence={}, show_progress=False
      ))

+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore      |   phi(internation_travel_control_singapore) |
+===========================================+=============================================+
| internation_travel_control_singapore(2.0) |                                      0.1273 |
+-------------------------------------------+---------------------------------------------+
| internation_travel_control_singapore(3.0) |                                      0.8727 |
+-------------------------------------------+---------------------------------------------+


In [32]:
print(ve.query(
          ["shn_requirement_singapore"], evidence={}, show_progress=False
      ))

+------------------------------+----------------------------------+
| shn_requirement_singapore    |   phi(shn_requirement_singapore) |
+==============================+==================================+
| shn_requirement_singapore(1) |                           0.9293 |
+------------------------------+----------------------------------+
| shn_requirement_singapore(2) |                           0.0707 |
+------------------------------+----------------------------------+


In [33]:
print(ve.query(
          ["shn_requirement_singapore"], evidence={"cases_per_mil_cat_malaysia": 1}, show_progress=False
      ))

+------------------------------+----------------------------------+
| shn_requirement_singapore    |   phi(shn_requirement_singapore) |
+==============================+==================================+
| shn_requirement_singapore(1) |                           1.0000 |
+------------------------------+----------------------------------+
| shn_requirement_singapore(2) |                           0.0000 |
+------------------------------+----------------------------------+


In [34]:
print(ve.query(
          ["shn_requirement_singapore"], evidence={"cases_per_mil_cat_malaysia": 2}, show_progress=False
      ))

+------------------------------+----------------------------------+
| shn_requirement_singapore    |   phi(shn_requirement_singapore) |
+==============================+==================================+
| shn_requirement_singapore(1) |                           1.0000 |
+------------------------------+----------------------------------+
| shn_requirement_singapore(2) |                           0.0000 |
+------------------------------+----------------------------------+


In [35]:
print(ve.query(
          ["shn_requirement_singapore"], evidence={"cases_per_mil_cat_malaysia": 4}, show_progress=False
      ))

+------------------------------+----------------------------------+
| shn_requirement_singapore    |   phi(shn_requirement_singapore) |
+==============================+==================================+
| shn_requirement_singapore(1) |                           1.0000 |
+------------------------------+----------------------------------+
| shn_requirement_singapore(2) |                           0.0000 |
+------------------------------+----------------------------------+


In [37]:
print(ve.query(
          ["shn_requirement_singapore"], evidence={"cases_per_mil_cat_malaysia": 5}, show_progress=False
      ))

+------------------------------+----------------------------------+
| shn_requirement_singapore    |   phi(shn_requirement_singapore) |
+==============================+==================================+
| shn_requirement_singapore(1) |                           1.0000 |
+------------------------------+----------------------------------+
| shn_requirement_singapore(2) |                           0.0000 |
+------------------------------+----------------------------------+
